In [54]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import os

tickers = ['SPY', 'IWM', 'DIA']
VERSION_NAME = 'widerATR_20day'
os.makedirs(f'backtesting_{VERSION_NAME}', exist_ok=True)

# Prepare Results + Technical Indicators

In [55]:
for ticker in tickers:
    # Combine the results from 2019 to 2024
    ticker_df = pd.DataFrame()
    for year in range(2019, 2024):
        df = pd.read_csv(f'./results_{VERSION_NAME}/{ticker}_{year}_{year+1}.csv')
        df['Date'] = pd.to_datetime(df['Date'])
        if year < 2023:
            # Testing on 2 years but since it overlaps, just take the first year except 2023-2024 take together
            df = df[df['Date'].dt.year == year]
        ticker_df = pd.concat([ticker_df, df], ignore_index=True)
    ticker_df = ticker_df.drop_duplicates()


    # Get technical indicators from yfinance
    data = yf.download(ticker, start='2018-01-01', end='2024-12-31')
    data.columns = [col[0] for col in data.columns]
    data.ta.sma(length=50, append=True)
    data.ta.sma(length=200, append=True)
    data.ta.ema(length=12, append=True)
    data.ta.ema(length=26, append=True)
    data.ta.ema(length=50, append=True)
    data.ta.rsi(append=True)
    data.ta.bbands(append=True)
    data.ta.macd(append=True)
    data.drop(columns=['Close'], inplace=True)

    # Merge results with indicators
    ticker_df.set_index('Date', inplace=True)
    combined_df = ticker_df.merge(data, left_index=True, right_index=True, how='inner')

    combined_df.to_csv(f'./backtesting_{VERSION_NAME}/{ticker}_combined.csv', index=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [56]:
# Continue from your merging code
def backtest_strategy(df, strategy_name, signal_col='Prediction', initial_capital=10000, commission=1):
    """Backtest a single strategy"""
    capital = initial_capital
    position = 0
    equity_curve = []
    trades = []
    
    for i, row in df.iterrows():
        current_price = row['Close']
        
        # Strategy-specific signal interpretation
        if strategy_name == 'CNN':
            signal = row[signal_col]  # 0=Down, 1=Neutral, 2=Up
        elif strategy_name == 'RSI':
            signal = 2 if row['RSI_14'] < 30 else (0 if row['RSI_14'] > 70 else 1)
        elif strategy_name == 'MACD':
            signal = 2 if row['MACD_12_26_9'] > row['MACDs_12_26_9'] else (0 if row['MACD_12_26_9'] < row['MACDs_12_26_9'] else 1)
        elif strategy_name == 'SMA':
            signal = 2 if row['SMA_50'] > row['SMA_200'] else (0 if row['SMA_50'] < row['SMA_200'] else 1)
        elif strategy_name == 'EMA_12_26':  # New EMA crossover strategy
            signal = 2 if row['EMA_12'] > row['EMA_26'] else (0 if row['EMA_12'] < row['EMA_26'] else 1)
        elif strategy_name == 'EMA_50':  # Price vs EMA strategy
            signal = 2 if current_price > row['EMA_50'] else (0 if current_price < row['EMA_50'] else 1)
        elif strategy_name == 'BB':
            if current_price > row['BBM_5_2.0']:
                signal = 2
            elif current_price < row['BBM_5_2.0']:
                signal = 0
            elif current_price < row['BBL_5_2.0'] and row['BBP_5_2.0'] < 0.2:
                signal = 2
            elif current_price > row['BBU_5_2.0'] and row['BBP_5_2.0'] > 0.8:
                signal = 0
            else:
                signal = 1
        elif strategy_name == 'Buy & Hold':
            if i == df.index[0]:
                position = initial_capital // current_price
                capital -= position * current_price + commission
            signal = 1  # Never sell
            
        # Execute trades
        if signal == 2 and position == 0:  # Buy signal
            shares = capital // current_price
            if shares > 0:
                capital -= shares * current_price + commission
                position = shares
                trades.append({'date': i, 'type': 'buy', 'price': current_price})
        elif signal == 0 and position > 0:  # Sell signal
            capital += position * current_price - commission
            trades.append({'date': i, 'type': 'sell', 'price': current_price})
            position = 0
            
        # Record daily portfolio value
        equity_curve.append({
            'Date': i,
            'Value': capital + position * current_price,
            'Position': position,
            'Price': current_price
        })
    
    equity_df = pd.DataFrame(equity_curve).set_index('Date')
    trades_df = pd.DataFrame(trades) if trades else pd.DataFrame()
    
    return equity_df, trades_df

def calculate_performance(equity_df, trades_df, initial_capital=10000):
    """Calculate performance metrics"""
    returns = equity_df['Value'].pct_change().dropna()
    peak = equity_df['Value'].cummax()
    drawdown = (equity_df['Value'] - peak) / peak
    
    if not trades_df.empty:
        winning_trades = trades_df[trades_df['type'] == 'sell']['price'] > trades_df[trades_df['type'] == 'sell']['price'].shift(1)
        win_rate = winning_trades.mean() if len(winning_trades) > 0 else 0
    else:
        win_rate = 0
    
    return {
        'Final Value': equity_df['Value'].iloc[-1],
        'Total Return (%)': (equity_df['Value'].iloc[-1] / initial_capital - 1) * 100,
        'Annualized Return (%)': ((equity_df['Value'].iloc[-1] / initial_capital) ** (1/((equity_df.index[-1] - equity_df.index[0]).days/365)) - 1) * 100,
        'Sharpe Ratio': (returns.mean() / returns.std()) * np.sqrt(252),
        'Max Drawdown (%)': drawdown.min() * 100,
        'Win Rate (%)': win_rate * 100,
        'Number of Trades': len(trades_df) // 2 if not trades_df.empty else 0
    }

def run_comparison(ticker):
    """Run full comparison for a single ticker"""
    # Load merged data
    df = pd.read_csv(f'./backtesting_{VERSION_NAME}/{ticker}_combined.csv', parse_dates=['Date'], index_col='Date')
    
    strategies = ['CNN', 'RSI', 'MACD', 'SMA', 'EMA_12_26', 'EMA_50', 'BB', 'Buy & Hold']
    results = {}
    equity_curves = {}
    
    for strategy in strategies:
        equity_df, trades_df = backtest_strategy(df, strategy)
        results[strategy] = calculate_performance(equity_df, trades_df)
        equity_curves[strategy] = equity_df['Value']
        
        # Save equity curves and trades
        # equity_df.to_csv(f'./backtesting_{VERSION_NAME}/{ticker}_{strategy}_equity.csv')
        # if not trades_df.empty:
        #     trades_df.to_csv(f'./backtesting_{VERSION_NAME}/{ticker}_{strategy}_trades.csv')
    
    # Plot equity curves
    plt.figure(figsize=(9, 4))
    for strategy, curve in equity_curves.items():
        plt.plot(curve.index, curve, label=strategy, alpha=0.7)
    plt.title(f'{ticker} Strategy Performance Comparison (2019-2024)')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value ($)')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'./backtesting_{VERSION_NAME}/{ticker}_performance.png')
    plt.close()
    
    # Save results
    results_df = pd.DataFrame.from_dict(results, orient='index')
    results_df.to_csv(f'./backtesting_{VERSION_NAME}/{ticker}_results.csv')
    
    return results_df

# Main execution loop
tickers = tickers
all_results = {}

for ticker in tickers:
    print(f"Processing {ticker}...")
    ticker_results = run_comparison(ticker)
    all_results[ticker] = ticker_results

# Generate consolidated report
consolidated = pd.concat(all_results, names=['Ticker', 'Strategy'])
consolidated.to_csv(f'./backtesting_{VERSION_NAME}/consolidated_results.csv')

print('Done!')

Processing SPY...
Processing IWM...
Processing DIA...
Done!


In [57]:
consolidated

Final Value  Total Return (%)  Annualized Return (%)  \
Ticker Strategy                                                            
SPY    CNN         18518.642975         85.186430              10.820958   
       RSI         13994.526489         39.945265               5.763906   
       MACD        18373.685425         83.736854              10.675840   
       SMA         19460.084930         94.600849              11.741064   
       EMA_12_26   19988.110840         99.881108              12.240999   
       EMA_50      16366.610626         63.666106               8.561562   
       BB          12294.314072         22.943141               3.504099   
       Buy & Hold  25817.046997        158.170470              17.133860   
IWM    CNN         19800.413139         98.004131              12.064562   
       RSI         12424.102142         24.241021               3.685497   
       MACD        16613.019897         66.130199               8.832402   
       SMA          7623.336472        -23.766635              -4.424068   
       EMA_12_26   11506.167465         15.061675               2.366948   
       EMA_50      12985.900322         29.859003               4.452936   
       BB          15005.450356         50.054504               7.001104   
       Buy & Hold  17669.624390         76.696244               9.957125   
DIA    CNN         16656.689453         66.566895               8.880052   
       RSI         14236.502884         42.365029               6.066662   
       MACD        17474.145615         74.741456               9.753349   
       SMA         13755.719101         37.557191               5.460808   
       EMA_12_26   13994.502060         39.945021               5.763875   
       EMA_50      14956.619644         49.566196               6.942965   
       BB          14491.859512         44.918595               6.381543   
       Buy & Hold  20430.347168        104.303472              12.651310   

                   Sharpe Ratio  Max Drawdown (%)  Win Rate (%)  \
Ticker Strategy                                                   
SPY    CNN             0.820652        -19.360874     66.666667   
       RSI             0.434860        -27.796387     80.000000   
       MACD            0.947289        -15.024496     75.757576   
       SMA             0.773645        -33.691554     50.000000   
       EMA_12_26       1.017995        -12.949981     77.777778   
       EMA_50          0.743134        -22.151291     50.000000   
       BB              0.349199        -26.815012     63.546798   
       Buy & Hold      0.899453        -33.685484      0.000000   
IWM    CNN             0.730370        -34.165668     53.153153   
       RSI             0.282902        -35.894480     40.000000   
       MACD            0.590705        -30.459841     63.333333   
       SMA            -0.144637        -44.119088     40.000000   
       EMA_12_26       0.222981        -33.302374     59.259259   
       EMA_50          0.347059        -35.554765     50.746269   
       BB              0.487146        -27.952269     56.345178   
       Buy & Hold      0.499545        -40.895886      0.000000   
DIA    CNN             0.593628        -31.054902     61.000000   
       RSI             0.459029        -30.390162     33.333333   
       MACD            0.920560        -13.389866     73.333333   
       SMA             0.416653        -36.227845     50.000000   
       EMA_12_26       0.550952        -12.848293     63.636364   
       EMA_50          0.658947        -16.581639     69.642857   
       BB              0.606406        -19.126218     59.473684   
       Buy & Hold      0.712340        -36.496539      0.000000   

                   Number of Trades  
Ticker Strategy                      
SPY    CNN                       78  
       RSI                        5  
       MACD                      66  
       SMA                        2  
       EMA_12_26                 18  
       EMA_50                    5